In [4]:
import pyltr
import pandas as pd
import numpy as np
import MRR
import csv
import time
import dawg
import pickle
import gc


In [29]:
num_training_lines = 106161931
num_validation_lines = 102964961
amount_of_divisions = 20

preallocate_training = int(num_training_lines / (amount_of_divisions))
preallocate_validation = int(num_validation_lines / (amount_of_divisions))

In [30]:
# Maak klaar voor andere feature krijgen

with open('../data/sorted_popular_queries.csv', mode='r') as infile:
    reader = csv.reader(infile)
    skipheader = next(reader)
    data=[tuple([line[1], int(line[2])]) for line in reader if int(line[2])>2]
sortedpopulardict = dawg.IntCompletionDAWG(data)

KeyboardInterrupt: 

In [ ]:
######## Reading possible candidates
disable_synthetic = False

In [ ]:
# training_data = pd.read_csv("../data/training_feature_data.csv", header=None)
training_file = "../data/training_feature_data.csv"
start_time = time.time()
training_ngram_features = []
training_ylabels = []
training_qids = []

training_ngram_features=preallocate_training*[[0,0,0,0,0,0]]
training_ylabels=preallocate_training*[np.bool(0)]
training_qids=preallocate_training*[np.int(0)]
training_popular_count_feature = preallocate_training*[np.int(0)]
training_word_length_feature = preallocate_training*[np.int(0)]
training_char_length_feature = preallocate_training*[np.int(0)]
training_lastspace_feature=preallocate_training*[np.bool(0)]

i = 0
with open(training_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    prev_qid = -1
    current_qid = -1
    split_counter = 0
    
    for row in spamreader:
        if int(row[3]) % amount_of_divisions == 0:
            continue
            
        # Skip if not in popular queries
        if disable_synthetic:
            if sortedpopulardict.get(row[2], 0) == 0:
                continue
            
        # Get features
        training_ngram_features[i] = eval(row[0])
        training_popular_count_feature[i] = sortedpopulardict.get(row[2], 0)
        training_word_length_feature[i] = len(row[2].split())
        training_char_length_feature[i] = len(row[2])
        training_lastspace_feature[i] = row[2][training_char_length_feature[i] - 1] == ' '

        # Set question and label
        training_ylabels[i] = np.bool(int(row[1]))
        training_qids[i] = int(row[3])
        i = i + 1
        if i == preallocate_training:
            break
            
        if i % 1000000 == 0:
            print(str(i) + "/ " + str(preallocate_training))
        
            
training_ngram_ints = np.array(training_ngram_features)
training_ylabels_ready = np.array(training_ylabels)
training_qids_ready = np.array(training_qids)

training_ngram_features = []
training_ylabels = []
training_qids = []

print("Dit koste: " + str(time.time() - start_time))

In [ ]:
# validation_data = pd.read_csv("../data/validation_feature_data.csv", header=None)
validation_file = "../data/validation_feature_data.csv"
start_time = time.time()
validation_ngram_features = []
validation_ylabels = []
validation_qids = []

validation_ngram_features=preallocate_validation*[[0,0,0,0,0,0]]
validation_ylabels=preallocate_validation*[np.bool(0)]
validation_qids=preallocate_validation*[np.int(0)]
validation_popular_count_feature = preallocate_validation*[np.int(0)]
validation_word_length_feature = preallocate_validation*[np.int(0)]
validation_char_length_feature = preallocate_validation*[np.int(0)]
validation_lastspace_feature=preallocate_validation*[np.bool(0)]

i = 0
with open(validation_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    prev_qid = -1
    current_qid = -1
    split_counter = 0
    
    for row in spamreader:
        if int(row[3]) % amount_of_divisions == 0:
            continue
        
        # Skip if not in popular queries
        if disable_synthetic:
            if sortedpopulardict.get(row[2], 0) == 0:
                continue
                
        # Get features
        validation_ngram_features[i] = eval(row[0])
        validation_popular_count_feature[i] = sortedpopulardict.get(row[2], 0)
        validation_word_length_feature[i] = len(row[2].split())
        validation_char_length_feature[i] = len(row[2])
        validation_lastspace_feature[i] = row[2][validation_char_length_feature[i] - 1] == ' '

        # Set question and label
        validation_ylabels[i] = np.bool(int(row[1]))
        validation_qids[i] = int(row[3])
        i = i + 1
        if i == preallocate_validation:
            break
        
        if i % 1000000 == 0:
            print(str(i) + "/ " + str(preallocate_validation))
            
validation_ngram_ints = np.array(validation_ngram_features)
validation_ylabels_ready = np.array(validation_ylabels)
validation_qids_ready = np.array(validation_qids)

validation_ngram_features = []
validation_ylabels = []
validation_qids = []

print("Dit koste: " + str(time.time() - start_time))

In [9]:
# # testing_data = pd.read_csv("../data/testing_feature_data.csv", header=None)
testing_file = "../data/testing_feature_data.csv"
start_time = time.time()
testing_ngram_features = []
testing_ylabels = []
testing_qids = []
preallocate_testing = 26821328
testing_ngram_features=preallocate_testing*[[0,0,0,0,0,0]]
testing_ylabels=preallocate_testing*[np.bool(0)]
testing_qids=preallocate_testing*[np.int(0)]
testing_popular_count_feature = preallocate_testing*[np.int(0)]
testing_word_length_feature = preallocate_testing*[np.int(0)]
testing_char_length_feature = preallocate_testing*[np.int(0)]
testing_lastspace_feature=preallocate_testing*[np.bool(0)]

i = 0
with open(testing_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    prev_qid = -1
    current_qid = -1
    split_counter = 0
    
    for row in spamreader:
        # Get features
        
        # Skip if not in popular queries
        if disable_synthetic:
            if sortedpopulardict.get(row[2], 0) == 0:
                continue
        
        testing_ngram_features[i] = eval(row[0])
        testing_popular_count_feature[i] = sortedpopulardict.get(row[2], 0)
        testing_word_length_feature[i] = len(row[2].split())
        testing_char_length_feature[i] = len(row[2])
        testing_lastspace_feature[i] = row[2][testing_char_length_feature[i] - 1] == ' '

        # Set question and label
        testing_ylabels[i] = np.bool(int(row[1]))
        testing_qids[i] = int(row[3])
        i = i + 1
        if i % 1000000 == 0:
            print(str(i) + "/ " + str(preallocate_validation))
            
testing_ngram_ints = np.array(testing_ngram_features)
testing_ylabels_ready = np.array(testing_ylabels)
testing_qids_ready = np.array(testing_qids)

testing_ngram_features = []
testing_ylabels = []
testing_qids = []

print("Dit koste: " + str(time.time() - start_time))



1000000/ 5148248
2000000/ 5148248
3000000/ 5148248
4000000/ 5148248
5000000/ 5148248
6000000/ 5148248
7000000/ 5148248
8000000/ 5148248
9000000/ 5148248
10000000/ 5148248
11000000/ 5148248
12000000/ 5148248
13000000/ 5148248
14000000/ 5148248
15000000/ 5148248
16000000/ 5148248
17000000/ 5148248
18000000/ 5148248
19000000/ 5148248
20000000/ 5148248
21000000/ 5148248
22000000/ 5148248
23000000/ 5148248
24000000/ 5148248
Dit koste: 353.3374288082123


In [19]:
### Select the features
ngram_features = True

In [20]:
# Merging all the features, ngram, word length, char length, popularity & last space

if ngram_features:
    training_features_final = np.ndarray(shape=(len(training_ngram_ints), 10))
    training_features_final[:,0:6] = training_ngram_ints
    training_features_final[:,6] = training_popular_count_feature
    training_features_final[:,7] = training_word_length_feature
    training_features_final[:,8] = training_char_length_feature
    training_features_final[:,9] = training_lastspace_feature
else:
    training_features_final = np.ndarray(shape=(len(training_ngram_ints), 4))
    training_features_final[:,0] = training_popular_count_feature
    training_features_final[:,1] = training_word_length_feature
    training_features_final[:,2] = training_char_length_feature
    training_features_final[:,3] = training_lastspace_feature


In [ ]:
# Merging all the features, ngram, word length, char length, popularity & last space

if ngram_features:
    validation_features_final = np.ndarray(shape=(len(validation_ngram_ints), 10))
    validation_features_final[:,0:6] = validation_ngram_ints
    validation_features_final[:,6] = validation_popular_count_feature
    validation_features_final[:,7] = validation_word_length_feature
    validation_features_final[:,8] = validation_char_length_feature
    validation_features_final[:,9] = validation_lastspace_feature
else:
    validation_features_final = np.ndarray(shape=(len(validation_ngram_ints), 4))
    validation_features_final[:,0] = validation_popular_count_feature
    validation_features_final[:,1] = validation_word_length_feature
    validation_features_final[:,2] = validation_char_length_feature
    validation_features_final[:,3] = validation_lastspace_feature


In [21]:
# Merging all the features, ngram, word length, char length, popularity & last space

if ngram_features:
    testing_features_final = np.ndarray(shape=(len(testing_ngram_ints), 10))
    testing_features_final[:,0:6] = testing_ngram_ints
    testing_features_final[:,6] = testing_popular_count_feature
    testing_features_final[:,7] = testing_word_length_feature
    testing_features_final[:,8] = testing_char_length_feature
    testing_features_final[:,9] = testing_lastspace_feature
else:
    testing_features_final = np.ndarray(shape=(len(testing_ngram_ints), 4))
    testing_features_final[:,0] = testing_popular_count_feature
    testing_features_final[:,1] = testing_word_length_feature
    testing_features_final[:,2] = testing_char_length_feature
    testing_features_final[:,3] = testing_lastspace_feature


In [22]:
######################## Model training - Currently, validation is disabled

In [23]:
metric = pyltr.metrics.ERR(k=10, highest_score = 1)
metric_MRR = MRR.MRR(k=10, highest_score = 1)

# Only needed if you want to perform validation (early stopping & trimming)
# monitor = pyltr.models.monitors.ValidationMonitor(
#     validation_features_final, validation_ylabels_ready, validation_qids_ready, metric=metric, stop_after=5)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=5,
    verbose=1
)

# model.fit(training_features_final, training_ylabels_ready, training_qids_ready, monitor=monitor)
model.fit(training_features_final, training_ylabels_ready, training_qids_ready)

 Iter  Train score    Remaining                           Monitor Output 
    1       0.1965       10.22m                                         
    2       0.1981        7.77m                                         
    3       0.1984        5.21m                                         
    4       0.1991        2.68m                                         
    5       0.2000        0.00s                                         


In [24]:
model.estimators_

array([[DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=True,
           random_state=<mtrand.RandomState object at 0x7f94a6cb71b0>,
           splitter='best')],
       [DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=True,
           random_state=<mtrand.RandomState object at 0x7f94a6cb71b0>,
           splitter='best')],
       [DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_

In [25]:
pickle.dump( model, open( "model1top100k5iter_nongram.p", "wb" ) )

In [26]:
Epred = model.predict(testing_features_final)
print('Our model:' +  str(metric.calc_mean(testing_qids_ready, testing_ylabels_ready, Epred)))

Our model:0.22471519880928417


In [27]:
print(' Our model MRR:'+  str(metric_MRR.calc_mean(testing_qids_ready, testing_ylabels_ready, Epred)))

 Our model MRR:0.4414690206869351


In [28]:
training_features_final

array([[4.803310e+06, 1.940000e+02, 5.200000e+01, ..., 3.000000e+00,
        1.300000e+01, 0.000000e+00],
       [4.802758e+06, 6.300000e+01, 1.900000e+01, ..., 3.000000e+00,
        1.400000e+01, 0.000000e+00],
       [4.802865e+06, 3.900000e+01, 1.600000e+01, ..., 3.000000e+00,
        1.500000e+01, 0.000000e+00],
       ...,
       [2.759820e+05, 3.320000e+02, 0.000000e+00, ..., 5.000000e+00,
        1.800000e+01, 0.000000e+00],
       [2.059000e+04, 2.142000e+03, 0.000000e+00, ..., 2.000000e+00,
        1.100000e+01, 0.000000e+00],
       [2.620700e+04, 5.030000e+02, 0.000000e+00, ..., 2.000000e+00,
        1.300000e+01, 0.000000e+00]])